<a href="https://colab.research.google.com/github/varunvenkatsaiis/colab_notebooks/blob/main/using_word_embeddings_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2025-03-22 09:29:39--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  12.9MB/s    in 9.3s    

2025-03-22 09:29:48 (8.64 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [2]:
!tar -xf aclImdb_v1.tar.gz

In [3]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [4]:
cd aclImdb/

/content/aclImdb


In [5]:
ls


imdbEr.txt  imdb.vocab  README  test/  train/


In [6]:
cd ..

/content


In [7]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [8]:
cd aclImdb/

/content/aclImdb


In [9]:
cd test/

/content/aclImdb/test


In [10]:
ls


labeledBow.feat  neg/  pos/  urls_neg.txt  urls_pos.txt


In [11]:
cd ../..

/content


In [12]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [13]:
cd aclImdb/

/content/aclImdb


In [14]:
cd train/

/content/aclImdb/train


In [15]:
ls


labeledBow.feat  neg/  pos/  unsup/  unsupBow.feat  urls_neg.txt  urls_pos.txt  urls_unsup.txt


In [16]:
cd ../../..

/


In [17]:
ls

bin@                        dev/     lib32@   NGC-DL-CONTAINER-LICENSE  root/  tmp/
boot/                       etc/     lib64@   opt/                      run/   tools/
content/                    home/    libx32@  proc/                     sbin@  usr/
cuda-keyring_1.1-1_all.deb  kaggle/  media/   python-apt/               srv/   var/
datalab/                    lib@     mnt/     python-apt.tar.xz*        sys/


In [18]:
cd content/

/content


In [19]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [20]:
!rm -r aclImdb/train/unsup

In [21]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [22]:
import os , pathlib , shutil , random


In [23]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("pos" , "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples : ]
  for fname in val_files:
    shutil.move( train_dir / category / fname ,
                val_dir / category / fname)



In [24]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [25]:
cd aclImdb/

/content/aclImdb


In [26]:
ls


imdbEr.txt  imdb.vocab  README  test/  train/  val/


In [27]:
cd val/

/content/aclImdb/val


In [28]:
ls

neg/  pos/


In [29]:
cd ../../

/content


In [30]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [31]:
from tensorflow import keras

In [32]:
batch_size = 32

In [33]:
train_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/train" , batch_size = batch_size
)



Found 20000 files belonging to 2 classes.


In [56]:
type(train_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [34]:
val_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/val" , batch_size = batch_size
)

Found 5000 files belonging to 2 classes.


In [35]:
test_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/test" , batch_size = batch_size
)

Found 25000 files belonging to 2 classes.


In [36]:
print(val_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [37]:
val_ds.class_names

['neg', 'pos']

In [38]:
from tensorflow.keras.layers import  TextVectorization

In [39]:
text_vectorization =  TextVectorization(
    max_tokens = 20000,
    output_mode = "multi_hot"

)

In [40]:
text_only_train_ds = train_ds.map(lambda x , y : x)

In [41]:
 text_vectorization.adapt(text_only_train_ds)

In [42]:
bin_1gram_train_ds = train_ds.map(
    lambda x , y : ( text_vectorization(x) , y) , num_parallel_calls = 4
)



In [43]:
bin_1gram_val_ds = val_ds.map(
    lambda x , y : ( text_vectorization(x) , y ) , num_parallel_calls = 4
)

In [44]:
bin_1gram_test_ds   = test_ds.map(
    lambda x, y : ( text_vectorization(x) , y ) , num_parallel_calls = 4

)

In [45]:
from tensorflow.keras import layers

In [46]:
def get_model( maxtokens = 20000 , hidden_dim = 16 ):
  inputs= keras.Input(shape=(maxtokens,))
  x = layers.Dense( hidden_dim , activation="relu"  )  (inputs)
  x = layers.Dropout( 0.5  )(x)
  outputs = layers.Dense( 1 , activation="sigmoid" ) (x)
  model = keras.Model( inputs , outputs )
  model.compile (  optimizer = "rmsprop" ,
                 loss = "binary_crossentropy",
                   metrics = ["accuracy"] )
  return model

In [47]:

model = get_model()
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram_model.keras" , save_best_only = True)

]

model.fit( bin_1gram_train_ds.cache() ,
          validation_data = bin_1gram_val_ds.cache()  ,
           epochs = 10,
           callbacks = callbacks)

model = keras.models.load_model("binary_1gram_model.keras")
print( model.evaluate(bin_1gram_test_ds ) )


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 17s 25ms/step - accuracy: 0.7705 - loss: 0.4887 - val_accuracy: 0.8870 - val_loss: 0.2791
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - accuracy: 0.8908 - loss: 0.2855 - val_accuracy: 0.8898 - val_loss: 0.2750
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9116 - loss: 0.2480 - val_accuracy: 0.8880 - val_loss: 0.2853
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9193 - loss: 0.2267 - val_accuracy: 0.8920 - val_loss: 0.3055
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9277 - loss: 0.2250 - val_accuracy: 0.8930 - val_loss: 0.3090
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9299 - loss: 0.2088 - val_accuracy: 0.8926 - val_loss: 0.3205
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.9375 - loss: 0.2044 - val_accuracy: 0.8906 - val_loss: 0.3303
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.9360 - loss: 0.2003 - val_accuracy

In [49]:
l = text_vectorization("A mind-bending thriller that masterfully blends dream manipulation with stunning visuals and an intricate narrative. Christopher Nolan’s direction, combined with Hans Zimmer’s score, creates an unforgettable cinematic experience.")

In [50]:
print(l)

tf.Tensor([1 0 1 ... 0 0 0], shape=(20000,), dtype=int64)


In [68]:
p_inputs = keras.Input( shape = (1,), dtype="string"  )
p_input = text_vectorization(p_inputs)
md = keras.models.load_model("binary_1gram_model.keras")
p_output = md( p_input )
inference_model = keras.Model(p_inputs , p_output)



In [69]:
import tensorflow as tf

raw_data = tf.convert_to_tensor( ["   A mind-bending thriller that masterfully blends dream manipulation with stunning visuals and an intricate narrative. Christopher Nolan’s direction, combined with Hans Zimmer’s score, creates an unforgettable cinematic experience."]
)



In [70]:
prediction = inference_model(raw_data)

In [71]:
prediction

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.9212185]], dtype=float32)>

In [74]:
float(prediction[0]*100)

92.1218490600586